# Winlogon Helper DLL
Winlogon.exe is a Windows component responsible for actions at logon/logoff as well as the secure attention sequence (SAS) triggered by Ctrl-Alt-Delete. Registry entries in HKLM\Software[Wow6432Node]Microsoft\Windows NT\CurrentVersion\Winlogon\ and HKCU\Software\Microsoft\Windows NT\CurrentVersion\Winlogon\ are used to manage additional helper programs and functionalities that support Winlogon. Malicious modifications to these Registry keys may cause Winlogon to load and execute malicious DLLs and/or executables.

## Rule Content
```
- title: Winlogon Helper DLL
  id: 851c506b-6b7c-4ce2-8802-c703009d03c0
  status: experimental
  description: Winlogon.exe is a Windows component responsible for actions at logon/logoff
    as well as the secure attention sequence (SAS) triggered by Ctrl-Alt-Delete. Registry
    entries in HKLM\Software[Wow6432Node]Microsoft\Windows NT\CurrentVersion\Winlogon\
    and HKCU\Software\Microsoft\Windows NT\CurrentVersion\Winlogon\ are used to manage
    additional helper programs and functionalities that support Winlogon. Malicious
    modifications to these Registry keys may cause Winlogon to load and execute malicious
    DLLs and/or executables.
  author: Timur Zinniatullin, oscd.community
  date: 2019/10/21
  modified: 2019/11/04
  references:
  - https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1004/T1004.yaml
  logsource:
    product: windows
    service: powershell
    description: Script block logging must be enabled
    category: null
  detection:
    selection:
      EventID: 4104
    keyword1:
    - '*Set-ItemProperty*'
    - '*New-Item*'
    keyword2:
    - '*CurrentVersion\Winlogon*'
    condition: selection and ( keyword1 and keyword2 )
  falsepositives:
  - Unknown
  level: medium
  tags:
  - attack.persistence
  - attack.t1004

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-powershell-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"4104" AND \*.keyword:(*Set\-ItemProperty* OR *New\-Item*) AND "*CurrentVersion\\Winlogon*")')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()